## 순환신경망 구현 및 학습

In [1]:
import tensorflow as tf

## 하이퍼 파라미터

In [2]:
EPOCHS = 10
NUM_WORDS = 10000

## 모델 정의

In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel,self).__init__()
        self.emb=tf.keras.layers.Embedding(NUM_WORDS,16) 
        self.rnn=tf.keras.layers.SimpleRNN(32)
        self.dense=tf.keras.layers.Dense(2,activation='softmax')
        
    def call(self,x,training=None,mask=None):
        x=self.emb(x)
        x=self.rnn(x)
        return self.dense(x)
        

## 학습, 테스트 루프 정의

In [4]:
# Implement training loop
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images, training=False)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## 데이터셋 준비


In [5]:
imdb=tf.keras.datasets.imdb
(x_train,y_train),(x_test,y_test)=imdb.load_data(num_words=NUM_WORDS)

#특정길이 잘라주고, 길이에 못 미치면 padding해줘야한다. 
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train,value=0,padding='pre',maxlen=32)
x_test=tf.keras.preprocessing.sequence.pad_sequences(x_test,value=0,padding='pre',maxlen=32)

train_ds=tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(32)
test_ds=tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [6]:
# Create model
model = MyModel()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 동작

In [7]:
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_seqs, test_labels in test_ds:
        test_step(model, test_seqs, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.5357742309570312, Accuracy: 71.60800170898438, Test Loss: 0.4480433166027069, Test Accuracy: 78.97599792480469
Epoch 2, Loss: 0.3706977367401123, Accuracy: 83.68000030517578, Test Loss: 0.502435564994812, Test Accuracy: 77.69200134277344
Epoch 3, Loss: 0.24011291563510895, Accuracy: 90.49199676513672, Test Loss: 0.5601006746292114, Test Accuracy: 76.32799530029297
Epoch 4, Loss: 0.1178683340549469, Accuracy: 95.96800231933594, Test Loss: 0.7556303143501282, Test Accuracy: 75.6719970703125
Epoch 5, Loss: 0.05171183496713638, Accuracy: 98.45600128173828, Test Loss: 0.9699835181236267, Test Accuracy: 74.62799835205078
Epoch 6, Loss: 0.026951590552926064, Accuracy: 99.15599822998047, Test Loss: 1.2016963958740234, Test Accuracy: 73.33200073242188
Epoch 7, Loss: 0.014141447842121124, Accuracy: 99.62000274658203, Test Loss: 1.3430951833724976, Test Accuracy: 75.01599884033203
Epoch 8, Loss: 0.015879372134804726, Accuracy: 99.49600219726562, Test Loss: 1.4521909952163696, Tes